In [1]:
%pip install --upgrade pip
# install dependencies and select solver
%pip install -q amplpy matplotlib pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
SOLVER = "cplex"

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["cplex"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Licensed to AMPL Community Edition License for <robert.cowlishaw.2017@uni.strath.ac.uk>.


In [3]:
import numpy as np
import csv
import os

In [57]:
# Load time dataset
SAVE_DIR = "data_icsmd_1day"

save_location = os.path.join(os.getcwd(), "..", "DATA", SAVE_DIR)
if not os.path.exists(save_location):
    raise Exception("Sat Data not setup, run get_active_tle.py first")

timer_all = np.load(save_location+"/all_combs.npy")

timer_all[np.isnan(timer_all)] = 0

timer_all = np.asanyarray(timer_all, dtype=int)



timer = timer_all[:,:,0:5]

print(np.shape(timer))

print(timer)

(10, 10, 5)
[[[    0     0     0     0     0]
  [64823     0     0     0     0]
  [    0     0     0     0     0]
  [    0     0     0     0     0]
  [62212 62242     0     0     0]
  [    0     0     0     0     0]
  [    0     0     0     0     0]
  [    0     0     0     0     0]
  [    0     0     0     0     0]
  [14855 14885 14915     0     0]]

 [[64823     0     0     0     0]
  [    0     0     0     0     0]
  [    0     0     0     0     0]
  [    0     0     0     0     0]
  [    0     0     0     0     0]
  [20827 85890 85920     0     0]
  [    0     0     0     0     0]
  [68304 68334 68364     0     0]
  [35892 35922 35952 35982 38773]
  [    0     0     0     0     0]]

 [[    0     0     0     0     0]
  [    0     0     0     0     0]
  [    0     0     0     0     0]
  [    0     0     0     0     0]
  [    0     0     0     0     0]
  [    0     0     0     0     0]
  [45136 45166     0     0     0]
  [14405 14435     0     0     0]
  [    0     0     0     0     0

In [99]:
%%writefile OBJ.mod

param N;

param dim1{i in 1..3};
# param dim2;

param T{i in 1..dim1[1], j in 1..dim1[2], k in 1..dim1[3]};

var x0 integer >= 0 <= N;
var x1 integer >= 0 <= N;
var x2 integer >= 0 <= N;
var x3 integer >= 0 <= N;

# param T_sum = sum {i in 1..dim1[1]} (max {j in 1..dim1[2]} T[i,j]);

# var init_x0_x1 = ;



maximize total_time: T[x0, x1, 0]; #max {i in T} i;

# s.t. stuff1: T[x0, x1, 0] > 0;

# s.t. stuff1: x0 <= N;

# s.t. germanium: x2 <= 1500;
# s.t. plastic: x1 + x2 <= 1750;
# s.t. copper: 4.04 * x1 + 2.02 * x2 <= 4800;

Overwriting OBJ.mod


In [100]:
m = AMPL()
m.read("OBJ.mod")

import numpy as np

timer = np.asanyarray(timer, dtype=int)
t_shape = np.shape(timer)

m.option["solver"] = SOLVER
m.param["N"] = 10
m.param["dim1"] = t_shape
# m.param["T"] = list(timer)
m.param["T"] = {(i+1, j+1, k+1): timer[i,j,k] for i in range(t_shape[0]) for j in range(t_shape[1]) for k in range(t_shape[2])}
# m.eval("option display_wdith 600;")
m.eval("display T;")

print("done")

# m.solve()

# print(f'x = ({m.var["x0"].value():.1f}, {m.var["x1"].value():.1f})')
# print(f'optimal value = {m.obj["total_time"].value():.2f}')

T [*,*,1]
:      1       2       3     4     5       6       7       8       9      10 :=
1        0   64823       0   0   62212       0       0       0       0   14855
2    64823       0       0   0       0   20827       0   68304   35892       0
3        0       0       0   0       0       0   45136   14405       0   35832
4        0       0       0   0       0       0       0       0       0       0
5    62212       0       0   0       0       0   25749   76527       0       0
6        0   20827       0   0       0       0   59000       0       0       0
7        0       0   45136   0   25749   59000       0       0     810   59361
8        0   68304   14405   0   76527       0       0       0   59871       0
9        0   35892       0   0       0       0     810   59871       0       0
10   14855       0   35832   0       0       0   59361       0       0       0

 [*,*,2]
:      1       2       3     4     5       6       7       8       9      10 :=
1        0       0       0   0

RuntimeError: Error executing "display" command:
error processing objective total_time:
	invalid subscript T[0,0,0]
